In [ ]:
import boto3

In [ ]:
pip install boto3

In [ ]:
import boto3

In [ ]:
import pandas as pd
import psycopg2
import json

In [ ]:
pip install psycopg2


In [ ]:
import pandas as pd
import psycopg2
import json

In [ ]:
import configparser
config = configparser.ConfigParser()
# Update path to a more secure or relative path
config.read_file(open('/path/to/your/config_file.config'))
# your config file

In [ ]:
config.get("AWS","KEY")

In [ ]:
KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE =config.get('DWH','DWH_CLUSTER_TYPE')
DWH_NUM_NODES = config.get('DWH','DWH_NUM_NODES')
DWH_NODE_TYPE = config.get('DWH','DWH_NODE_TYPE')
DWH_CLUSTER_IDENTIFIER = config.get('DWH','DWH_CLUSTER_IDENTIFIER')
DWH_DB = config.get('DWH','DWH_DB')
DWH_DB_USER = config.get('DWH','DWH_DB_USER')
DWH_DB_PASSWORD = config.get('DWH','DWH_DB_PASSWORD')
DWH_PORT = config.get('DWH','DWH_PORT')
DWH_IAM_ROLE_NAME = config.get('DWH','DWH_IAM_ROLE_NAME')

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

In [ ]:
pd.DataFrame({ "Param":
              ['DWH_CLUSTER_TYPE', 'DWH_NUM_NODES', 'DWH_NODE_TYPE', 'DWH_CLUSTER_IDENTIFIER',
                'DWH_DB', 'DWH_DB_USER', 'DWH_DB_PASSWORD', 'DWH_PORT', 'DWH_IAM_ROLE_NAME'],
              "Value":
              [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER,
              DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]   
})

In [ ]:
ec2 = boto3.resource('ec2',
                    region_name = "eu-central-1",
                    aws_access_key_id = KEY,
                    aws_secret_access_key = SECRET)

In [ ]:
s3 = boto3.resource('s3',
                    region_name = "eu-central-1",
                    aws_access_key_id = KEY,
                    aws_secret_access_key = SECRET)

In [ ]:
iam = boto3.client('iam',                   
                    aws_access_key_id = KEY,
                    aws_secret_access_key = SECRET,
                    region_name = "eu-central-1",
                    )

redshift = boto3.client('redshift',
                    region_name = "eu-central-1",
                    aws_access_key_id = KEY,
                    aws_secret_access_key = SECRET)

In [ ]:
bucket = s3.Bucket("first-cloud-project-dato")
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix = '')]
log_data_files

In [ ]:
roleArn = iam.get_role(RoleName = DWH_IAM_ROLE_NAME)['Role']['Arn']

In [ ]:
roleArn


In [ ]:
try:
    response = redshift.create_cluster(
    ClusterType = DWH_CLUSTER_TYPE,
        NodeType = DWH_NODE_TYPE,
        
        DBName = DWH_DB,
        ClusterIdentifier = DWH_CLUSTER_IDENTIFIER,
        MasterUsername = DWH_DB_USER,
        MasterUserPassword = DWH_DB_PASSWORD,
        
        IamRoles = [roleArn]
    )
except Exception as e:
    print(e)
    


In [ ]:
redshift.describe_clusters(ClusterIdentifier = DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', None)
    keysToShow = ["ClusterIdentifier", "NodeType","ClusterStatus", "MasterUsername",
                 "DBName", "Endpoint", "VpcId"]
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data = x, columns = ["key", "Value"])
myClusterProps = redshift.describe_clusters(ClusterIdentifier = DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

In [ ]:
prettyRedshiftProps(myClusterProps)

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
DB_NAME = myClusterProps['DBName']
DB_USER = myClusterProps['MasterUsername']

In [ ]:
DB_NAME


In [ ]:
try:
    vpc = ec2.Vpc(id = myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName = defaultSg.group_name,
        CidrIp = '0.0.0.0/0',
        IpProtocol = 'TCP',
        FromPort = int(DWH_PORT),
        ToPort = int(DWH_PORT)
    
    )
except Exception as e:
    print(e)

In [ ]:
try:
    conn = psycopg2.connect(host = DWH_ENDPOINT, dbname = DB_NAME, user = DB_USER, password = "Password123", port = 5439)
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
conn.set_session(autocommit = True)

In [ ]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)

In [ ]:
try:
    cur.execute(""" create table users(
    userid integer not null distkey sortkey,
    username char(8),
    firstname varchar(30),
    lastname varchar(30),
    city varchar(30),
    state char(2),
    email varchar(100),
    phone char(14),
    likesports boolean,
    liketheatre boolean,
    likeconcerts boolean,
    likejazz boolean,
    likeclassical boolean,
    likeopera boolean,
    likerock boolean,
    likevegas boolean,
    likesboradway boolean,
    likemusicals boolean); """)
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [ ]:
try:
    cur.execute(""" create table venue(
    venueid smallint not null distkey sortkey,
    venuename varchar(100),
    venuecity varchar(30),
    venuestate char(2),
    venueseats integer);""")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)
    

In [ ]:
try:
    cur.execute(""" create table category(
    catid smallint not null distkey sortkey,
    catgroup varchar(10),
    catname varchar(10),
    catedsc varchar(50));
    
    create table date(
    dateid smallint not null distkey sortkey,
    caldate date not null,
    day character(3) not null,
    week smallint not null,
    month character(5) not null,
    qtr character(5) not null,
    year smallint not null,
    holiday boolean default('N'));
    
    create table event(
    eventid integer not null distkey,
    venueid smallint not null,
    catid smallint not null,
    dateit smallint not null sortkey,
    eventname varchar(200),
    starttime timestamp);
    
    create table listing(
    listid integer not null distkey,
    sellerid integer not null,
    eventid integer not null,
    dateid smallint not null sortkey,
    numtickets smallint not null,
    priceperticket decimal(8,2),
    totalprice decimal(8,2),
    listtime timestamp);
    
    
    
    """)
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [ ]:
try:
    cur.execute("""
    copy users from 'your s3'
    credentials 'your role'
    region 'eu-central-1'

    """)
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [ ]:
s3://first-cloud-project-dato/category_pipe.txt

In [ ]:
try:
    cur.execute("""
        select * from users;

    """)
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)
    

In [ ]:
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

In [ ]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'limit_output': 1000000})

In [ ]:
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

In [ ]:
cur.execute("SELECT * FROM users")
rows = cur.fetchall()
print(rows)

In [ ]:
%config NotebookApp.iopub_data_rate_limit=10000000

In [ ]:
cur.execute("""
        SELECT * FROM users LIMIT 10;

    """)


In [ ]:
redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)